In [ ]:
import numpy as np
import json

def preprocess_gesture_json(json_path, sequence_length=10):
    with open(json_path, "r") as f:
        data = json.load(f)

    frames = []
    for frame in data:
        if "hands" in frame and frame["hands"]:
            landmarks = frame["hands"][0]
            if len(landmarks) != 21 or landmarks[0] == [0, 0]:
                continue

            wrist = landmarks[0]
            normalized = [(x - wrist[0], y - wrist[1]) if [x, y] != [0, 0] else (0.0, 0.0) for x, y in landmarks]
            flat = [val for pair in normalized for val in pair]
            frames.append(flat)

    if len(frames) < sequence_length:
        raise ValueError(f"❌ Không đủ frame hợp lệ. Cần ≥ {sequence_length}, hiện có {len(frames)}")

    # Lấy đoạn cuối chuỗi
    input_seq = np.array(frames[-sequence_length:])
    return input_seq.reshape(1, sequence_length, -1)


In [5]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# Load model & LabelEncoder
model = load_model("gesture_lstm_model.h5")
le = joblib.load("label_encoder.pkl")

# Load dữ liệu từ JSON mới
X_input = preprocess_gesture_json("json/qtest.hands.json", sequence_length=10)
print("✅ Input shape:", X_input.shape)

# Dự đoán
y_pred = model.predict(X_input)
pred_label = le.inverse_transform([np.argmax(y_pred)])

print("🧠 Cử chỉ dự đoán là:", pred_label[0])

✅ Input shape: (1, 10, 42)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
🧠 Cử chỉ dự đoán là: q
